In [7]:
import math
import string
import operator as ops

tests = """
3:10
5:30
5+10
8*3
20/4
932//7
5+8.3
2**8
2^8
2^^3
"""

In [3]:
class Range:
    def __init__(self, start, stop=1, step=1):
        self.start = start
        self.stop = stop
        self.step = step
        self.value = None
        self.report = ['start', 'stop', 'step']
        
    def evaluate(self):
        self.value = list(range(self.start, self.stop+1, self.step))
        self.value = Array(self.value)
        return self.value

In [4]:
def tetration(n, m):
    r = n
    for i in range(m):
        r = n ** r
    return r

tetration(2, 3)

65536

In [5]:
class Expression:
    def __init__(self, a, b, op):
        self.a = a
        self.b = b
        self.op = op
        self.op_list = {
            '+': ops.add,
            '*': ops.mul,
            '-': ops.sub,
            '/': ops.truediv,
            '//': ops.floordiv,
            '**': ops.pow,
            '^': ops.pow,
            '^^': tetration,
        }
        self.report = ['a', 'b']
        
    def evaluate(self):
        op_name = self.op_list[self.op]
        self.value = op_name(self.a, self.b)
        return self.value

In [6]:
class Token:
    def __init__(self, string='', token_type=None):
        self.string = string
        self.token_type = self.type = token_type
        
    def like(self, test):
        return self.token_type == test or self.token_type.startswith(test)

In [8]:
class Array:
    def __init__(self, terms):
        self.data = []
        for t in terms:
            if type(t) is list:
                self.data.append(array(t))
            else:
                if t not in list(',;'):
                    self.data.append(t)
                    
    def __str__(self):
        items = ', '.join(map(str, self.data))
        items = '['+items+']'
        return items

In [9]:
class Block:
    def __init__(self, components=None, parsed=None, parser=None, r=None):
        self.components = components
        self.parser = parser
        self.children = []
        
        self.parsed = None
        if self.components:
            comp_vals = []
            for q in self.components:
                if type(q) is Token:
                    q = q.string
                if q.isnumeric():
                    q = int(q)
                comp_vals.append(q)
            self.parsed = parser(*r(comp_vals))
            
    def add(self, x):
        self.children.append(x)
        
    def print(self, level=0):
        indent = ' '*2*level
        print(indent + str(self) + '; ' + type(self.parsed).__name__)
        for c in self.children:
            c.print(level=level+1)
#         for k, v in vars(self.parser).items():
#             print('{}{}:{}'.format(indent, k, v))
        if self.parsed:
            for k in self.parsed.report:
                v = getattr(self.parsed, k)
                print('{}{}:{}'.format(indent, k, str(v)))
        
    def __getitem__(self, i):
        return self.children[i]

In [10]:
class Program:
    def __init__(self, source):
        self.source = source
#         Create the root node
        self.tree = Block()
        
#         List of characters and their corresponding type
        self.char_sets = {
            'op': '!@#$%^&*/-+<>',
            'syntax': '()[]{},.;:=|',
            'letter': string.ascii_lowercase,
            'numeric': string.digits + '.-'
        }
#         List of syntactical patterns to match to generate the program's structure
#         i.e., the grammar
        self.patterns = {
#             'range': ['numeric', ':', 'numeric']
            'range': [
                lambda x: [x[0].like('num') and x[1] == ':' and x[2].like('num')],
                lambda x: [x[0], x[2]],
                Range
            ],
            'expression': [
                lambda x: x[1].like('op'),
                lambda x: [x[0], x[2], x[1]],
                Expression
            ]
        }
        
        self.parse()
        
    def char_type(self, x):
#         return list(filter(lambda x: k for k, v in self.char_sets if x in v))[0]
        return [k for k, v in self.char_sets.items() if x in v][0]
    
    def lex(self):
        statements = self.source.replace('|', '\n').split('\n')
#         Remove empty lines
        statements = list(filter(None, statements))
#         Loop through lines in program
        for s in statements:
            statement_parse = []
            token = Token(s[0], self.char_type(s[0]))
            for c in s[1:]:
                c_type = self.char_type(c)
#                 If character matches the type of the current token, append it
                if c_type == token.token_type:
                    token.string += c
#                 Otherwise, store the token and start a new one
                else:
                    statement_parse.append(token)
                    token = Token(c, c_type)
            statement_parse.append(token)
            self.tree.add(statement_parse)
    
    def parse(self):
        self.lex()
        
        for i, b in enumerate(self.tree.children):
            for k, v in self.patterns.items():
                pattern, result, block_type = v
                if pattern(b):
                    self.tree.children[i] = Block(b, parser=block_type, r=result)
                    
    def execute(self, node=None):
#         Default node is the root node
        if not node:
            node = self.tree
        
#         Loop through nodes in tree
        for b in node:
            if b.parser in [Range, Expression]:
                b.parsed.evaluate()
                print(b.parsed.value)
#             Recursively execute subnodes
            else:
#                 for v in b:
#                     self.execute()
               self.execute(node=b)
                    
    def print_tree(self):
        self.tree.print()
    
        
M = Program(tests)
M.execute()
vars(M.tree[0].parsed)
M.print_tree()
# M.tree[0].components[1].like('op')
# M.tree[-1].parser#.components[1].token_type

[3, 4, 5, 6, 7, 8, 9, 10]
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
15
24
5.0
133
13
256
256
65536
<__main__.Block object at 0x000001BDBADC1550>; NoneType
  <__main__.Block object at 0x000001BDBAE08C40>; Range
  start:3
  stop:10
  step:1
  <__main__.Block object at 0x000001BDBAD77E50>; Range
  start:5
  stop:30
  step:1
  <__main__.Block object at 0x000001BDBAD88730>; Expression
  a:5
  b:10
  <__main__.Block object at 0x000001BDBAD88F40>; Expression
  a:8
  b:3
  <__main__.Block object at 0x000001BDBAD88850>; Expression
  a:20
  b:4
  <__main__.Block object at 0x000001BDBAD88EE0>; Expression
  a:932
  b:7
  <__main__.Block object at 0x000001BDBAD88FA0>; Expression
  a:5
  b:8
  <__main__.Block object at 0x000001BDBAD88F70>; Expression
  a:2
  b:8
  <__main__.Block object at 0x000001BDBAD7E760>; Expression
  a:2
  b:8
  <__main__.Block object at 0x000001BDBAD7E340>; Expression
  a:2
  b:3


In [161]:
M.__dict__

{'source': '\n3:10\n5:30\n5+10\n8*3\n20/4\n932//7\n',
 'tree': <__main__.Block at 0x28a9c2ed940>,
 'char_sets': {'op': '!@#$%^&*/-+<>',
  'syntax': '()[]{},.;:=|',
  'letter': 'abcdefghijklmnopqrstuvwxyz',
  'numeric': '0123456789.-'},
 'patterns': {'range': [<function __main__.Program.__init__.<locals>.<lambda>(x)>,
   <function __main__.Program.__init__.<locals>.<lambda>(x)>,
   __main__.Range],
  'expression': [<function __main__.Program.__init__.<locals>.<lambda>(x)>,
   <function __main__.Program.__init__.<locals>.<lambda>(x)>,
   __main__.Expression]}}

In [124]:
bool([False])

True